In [1]:
import torch
import numpy as np
import os
os.chdir('../')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

N = 128
M = 128
K = 1

In [2]:
scale_dir = "/data/edm/scale/rbf_ecp_marginal_M=128"
os.makedirs(scale_dir, exist_ok=True)

In [6]:
root_dir = 'samples/edm-cifar10-32x32-uncond-vp/dpm_solver++_200'
noises = []
samples = []
for i in range(8):
    file = os.path.join(root_dir, f'samples_{i}.npz')
    data = np.load(file)
    print(data['noises'].shape, data['samples'].shape)
    noises.append(data['noises'])
    samples.append(data['samples'])

noises = torch.tensor(np.concatenate(noises, axis=0))[:N]
samples = torch.tensor(np.concatenate(samples, axis=0))[:N]
print(noises.shape, samples.shape)

(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
(16, 3, 32, 32) (16, 3, 32, 32)
torch.Size([128, 3, 32, 32]) torch.Size([128, 3, 32, 32])


In [7]:
import pickle
from samplers.utils import NoiseScheduleEDM, model_wrapper

device = "cuda" if torch.cuda.is_available() else "cpu"
# Fix the seed for z = sde.prior_sampling(shape).to(device) in deterministic sampling
torch.manual_seed(10)

# Load network.
ckp_path="/data/checkpoints/edm-cifar10-32x32-uncond-vp.pkl"
print(f'Loading network from "{ckp_path}"...')
with open(ckp_path, "rb") as f:
    net = pickle.load(f)["ema"].to(device)

ns = NoiseScheduleEDM()
ns

Loading network from "/data/checkpoints/edm-cifar10-32x32-uncond-vp.pkl"...


In [8]:
from samplers.rbf_ecp_marginal import RBFSolverECPMarginal
sigma_min=0.002
sigma_max=80

rbf = RBFSolverECPMarginal(ns, algorithm_type="data_prediction", scale_dir=scale_dir)
def rbf_sampler(model_fn, z, x, steps, order, skip_type='logSNR'):
    with torch.no_grad():
        pred = rbf.sample_by_target_matching(
            model_fn,
            z,
            x,
            steps=steps,
            t_start=sigma_max,
            t_end=sigma_min,
            order=order,
            skip_type=skip_type,
        )
        return pred

sampling_fn = rbf_sampler

In [9]:
import torch.nn.functional as F

for steps in [5, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40]:
    for order in [3, 4,]:
        for _ in range(K):
            indexes = np.random.randint(0, len(noises), size=(M,))
            zs = noises[indexes].to(device)
            xs = samples[indexes].to(device)
            zs = zs.to(torch.float64) * sigma_max
            noise_pred_fn = model_wrapper(net, ns, None)
            pred = sampling_fn(noise_pred_fn, zs, xs, steps, order)
            print(F.mse_loss(pred, xs))

def sample_by_target_matching start!!!
x.shape: torch.Size([128, 3, 32, 32]), target.shape: torch.Size([128, 3, 32, 32]), steps: 5, order: 3, skip_type: logSNR, lower_order_final: True
/data/edm/scale/rbf_ecp_marginal_M=128/NFE=5,p=3,number=0.npz  saved!
tensor(0.0234, device='cuda:0', dtype=torch.float64)
def sample_by_target_matching start!!!
x.shape: torch.Size([128, 3, 32, 32]), target.shape: torch.Size([128, 3, 32, 32]), steps: 5, order: 4, skip_type: logSNR, lower_order_final: True
/data/edm/scale/rbf_ecp_marginal_M=128/NFE=5,p=4,number=0.npz  saved!
tensor(0.0218, device='cuda:0', dtype=torch.float64)
def sample_by_target_matching start!!!
x.shape: torch.Size([128, 3, 32, 32]), target.shape: torch.Size([128, 3, 32, 32]), steps: 6, order: 3, skip_type: logSNR, lower_order_final: True
/data/edm/scale/rbf_ecp_marginal_M=128/NFE=6,p=3,number=0.npz  saved!
tensor(0.0174, device='cuda:0', dtype=torch.float64)
def sample_by_target_matching start!!!
x.shape: torch.Size([128, 3, 32, 32])

KeyboardInterrupt: 